In [16]:
import time
import warnings
from dotenv import load_dotenv
from langchain.tools import tool
MODEL_NAME = 'gemini-2.5-flash'
from typing import TypedDict, Annotated, Dict, Any
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

try:
    from langchain_tavily import TavilySearch
    from langchain.agents import create_react_agent, AgentExecutor
    from langchain import hub
except ImportError as e:
    print(f"Warning: Search tools not available: {e}")

warnings.filterwarnings("ignore")
load_dotenv()


True

In [17]:
@tool
def get_current_time():
    """Getting current date and time"""
    return time.ctime()

In [18]:
class StreamingSearchChain:
    def __init__(self):
        self.model = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0.7)

        try:
            self.search_tool = TavilySearch(max_results=5)
            print("✅ Search agent initialized")
        except Exception as e:
            print(f"⚠️ Search initialization failed: {e}")

    def execute_search_chain(self, query: str, update_callback=None):
        """Execute search chain: Search -> Stream -> Generate Response"""

        # Step 1: Show search query
        if update_callback:
            update_callback("query", f"🔍 Searching: {query}")

        try:
            # Step 2: Execute search tool
            if update_callback:
                update_callback("status", "🌐 Querying search engines...")

            # Get search results
            search_results_raw = self.search_tool.invoke({"query": query})
            print(search_results_raw)

            # Step 3: Process and stream sources
            sources = []
            search_results = []
            search_context = ""

            if isinstance(search_results_raw, list):
                for item in search_results_raw:
                    if isinstance(item, dict):
                        title = item.get("title", "No title")
                        url = item.get("url", "")
                        content = item.get("content", "")

                        # Store processed results
                        search_results.append(
                            {
                                "title": title,
                                "url": url,
                                "content": (
                                    content[:200] + "..."
                                    if len(content) > 200
                                    else content
                                ),
                                "score": item.get("score", 0),
                            }
                        )

                        # Build context for agent
                        search_context += (
                            f"Title: {title}\nURL: {url}\nContent: {content}\n\n"
                        )

                        if title != "No title":
                            source = f"{title}"
                            if url:
                                source += f" ({url})"
                            sources.append(source)

                            # Stream each source
                            if update_callback:
                                update_callback("source", source)
                                time.sleep(0.3)  # Streaming delay

            # Step 4: Generate response using search context
            if update_callback:
                update_callback("status", "🤖 Generating response...")

            # Create prompt with search context
            prompt = f"""
            Based on the following search results, provide a comprehensive answer to the query: "{query}"
            
            Search Results:
            {search_context}
            
            Please provide a well-structured answer based on this information.
            """

            response = self.model.invoke(prompt)
            agent_answer = response.content

            # Step 5: Show final answer
            if update_callback:
                update_callback("answer", agent_answer)

            return {
                "search_query": query,
                "agent_answer": agent_answer,
                "search_results": search_results,
                "sources": sources,
                "search_context": search_context,
                "error": None,
            }

        except Exception as e:
            error_msg = f"Search failed: {str(e)}"
            if update_callback:
                update_callback("error", error_msg)

            return {
                "search_query": query,
                "agent_answer": error_msg,
                "search_results": [],
                "sources": [],
                "search_context": "",
                "error": str(e),
            }

In [19]:
# State and Graph Setup
class AgentGraphState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    search_data: Dict[str, Any]

In [20]:
search_chain = StreamingSearchChain()

✅ Search agent initialized


In [21]:
def run_search_chain(state: AgentGraphState) -> AgentGraphState:
    messages = state["messages"]
    user_query = messages[-1].content

    print(f"\n🔍 Processing: '{user_query}'")

    # Execute the search chain (without callback for graph execution)
    search_data = search_chain.execute_search_chain(user_query)

    return {
        "messages": [AIMessage(content=search_data["agent_answer"])],
        "search_data": search_data,
    }

In [22]:
graph = StateGraph(AgentGraphState)
graph.add_node("search_chain", run_search_chain)

graph.add_edge(START, "search_chain")
graph.add_edge("search_chain", END)

compiled_graph = graph.compile(checkpointer=InMemorySaver())

In [23]:
messages = compiled_graph.invoke({'messages': 'ind vs eng'}, {'configurable': {'thread_id': '1'}})
messages


🔍 Processing: 'ind vs eng'
{'query': 'ind vs eng', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://timesofindia.indiatimes.com/sports/cricket/india-tour-of-england/after-ind-vs-eng-series-jasprit-bumrahs-whats-next-post-keeps-fans-wondering/articleshow/123122700.cms', 'title': "After IND vs ENG series, Jasprit Bumrah's 'what's next' post ...", 'content': 'India and England concluded a captivating Test series with a 2-2 draw. The final match saw India secure a narrow six-run victory at The Oval.', 'score': 0.7515939, 'raw_content': None}, {'url': 'https://sports.ndtv.com/england-vs-india-2025/india-vs-england-live-score-ind-vs-eng-5th-test-live-updates-day-5-scorecard-london-oval-rain-weather-update-mohammed-siraj-prasidh-krishna-akash-deep-9014970', 'title': 'India vs England Highlights, 5th Test Day 5: Mohammed ...', 'content': 'India vs England Highlights, 5th Test Day 5: Shubman Gill-led India have clinched a dramatic six-run victory over Eng

{'messages': [HumanMessage(content='ind vs eng', additional_kwargs={}, response_metadata={}, id='feb90a96-abce-4ce5-9e78-3cbf870b7bfe'),
  AIMessage(content='Based on the provided search results, which are empty, I am unable to provide a comprehensive answer regarding "IND vs ENG".\n\nThe search results section contains no information about matches, series, scores, or any related context between India and England in cricket (or any other sport).\n\nTo provide a comprehensive answer, I would typically need information such as:\n*   Recent match results\n*   Series schedules and formats (Tests, ODIs, T20Is)\n*   Player statistics and performances\n*   Historical rivalry information\n*   Current news or developments related to their encounters', additional_kwargs={}, response_metadata={}, id='f5637bb4-3840-4fc4-a32e-2a5b6a2a4a76')],
 'search_data': {'search_query': 'ind vs eng',
  'agent_answer': 'Based on the provided search results, which are empty, I am unable to provide a comprehensiv

In [24]:
messages['messages'][-1].content

'Based on the provided search results, which are empty, I am unable to provide a comprehensive answer regarding "IND vs ENG".\n\nThe search results section contains no information about matches, series, scores, or any related context between India and England in cricket (or any other sport).\n\nTo provide a comprehensive answer, I would typically need information such as:\n*   Recent match results\n*   Series schedules and formats (Tests, ODIs, T20Is)\n*   Player statistics and performances\n*   Historical rivalry information\n*   Current news or developments related to their encounters'